In [1]:
import ollama
import chromadb
import pandas as pd
import psycopg2
import pgvector
from pgvector.psycopg2 import register_vector
import json
import os
print()

In [2]:
# Connect to PostgreSQL database in Timescale using connection string
conn = psycopg2.connect(
    host=os.environ['HOST'],                        # e.g., "localhost"
    database=os.environ['DATABASE'],                # your database name
    user=os.environ['USER'],                        # your username
    password=os.environ['PASSWD'],                  # your password   
    port=os.environ['PORT']                         # the default port for PostgreSQL is 5432
)
cur = conn.cursor()

#install pgvector
cur.execute("CREATE EXTENSION IF NOT EXISTS vector");
conn.commit()    

In [3]:
# Register the vector type with psycopg2
register_vector(conn)

In [4]:
# Create table to store embeddings and metadata
table_create_command = """
CREATE TABLE IF NOT EXISTS public.vector_store4 (
            id uuid DEFAULT uuid_generate_v4() PRIMARY KEY,
            content text,
            metadata json,
            embedding vector(768)
            );
            """

cur.execute(table_create_command)
conn.commit()

In [5]:
# # Create table to store embeddings and metadata
# table_create_command = """
# DELETE FROM public.vector_store
#             """

# cur.execute(table_create_command)
# conn.commit()

In [ ]:
try:    
    select_query = """
    SELECT id, content, metadata, embedding
    FROM public.vector_store4;
    """

    # Execute the query
    cur.execute(select_query)

    # Fetch all rows from the table
    rows = cur.fetchall()

    # Print the results
    print("Data from the vector_store table:")
    for row in rows:
        print(f"ID: {row[0]}")
        print(f"Content: {row[1]}")
        print(f"Metadata: {json.dumps(row[2], indent=2)}")  # Pretty print the JSON metadata
        print(f"Embedding: {row[3][:10]}...")  # Print the first 10 elements of the embedding (1536-dimensional vector)
        print("="*50)

except psycopg2.DatabaseError as error:
    print(f"Error occurred: {error}")

Read CSV and Insert data

In [ ]:
df = pd.read_csv('tcc_ceds_music.csv')
df

In [ ]:
df.columns

In [17]:
def getEmbedding(row):
    # generate a response combining the prompt and data we retrieved in step 2
    summarize_lyrics = ollama.generate(
        model="llama3.1",
        # prompt=f"This is the prompt from the user: {data}. Respond using this information: {json.dumps(data_from_db)}. Only use the book from the database to answer.",
        prompt=f"""
        Summarize this random word using under 100 words: {str(row["lyrics"])}. 

        Do not write "Here's a summary", only write the content.
        """
    )
    print(summarize_lyrics['response'])

    data = {
        # "content": str(row["lyrics"]),
        "content": summarize_lyrics['response'],
        "metadata": {
                "artist_name": row["artist_name"],
                "track_name": row["track_name"],
                "release_date": str(row["release_date"]),
                "len": str(row["len"]),
                "genre": row["genre"],
                "topic": row["topic"]
                }
    } 
    
    prompt = f"""
    This is the metadata of the music: {json.dumps(data["metadata"])}
    
    This is the lyrics of the music: {data["content"]}

    """
    
    # print(prompt)
    # meta_embedding = ollama.embeddings(
    #     prompt=json.dumps(json.dumps(data["metadata"])),
    #     model="nomic-embed-text",
    #     options={"num_ctx": 8192}
    # )
    embedding = ollama.embeddings(
        prompt=prompt,
        model="nomic-embed-text",
        options={"num_ctx": 8192}
    )
    # embedding size 768
    return embedding["embedding"]

In [17]:
def addData2Table(index, cur, conn, row, collection, is_add_to_table=True):
    data = {
        "content": str(row["lyrics"]),
        "metadata": {
                "track_name": row["track_name"],
                "artist_name": row["artist_name"],
                "release_date": str(row["release_date"]),
                "len": str(row["len"]),
                "genre": row["genre"],
                "topic": row["topic"],
                },
        "embedding": getEmbedding(row)
    }
    # cur.execute("SELECT 1 FROM public.vector_store4 WHERE content = %s;", (data["content"],))
    # exists = cur.fetchone()
    # conn.commit()

    # if exists:
    
    
    if is_add_to_table:
        insert_query = """
            INSERT INTO public.vector_store4 (content, metadata, embedding)
            VALUES (%s, %s, %s::vector);
        """

        # Execute the query with provided values
        cur.execute(insert_query, (data["content"], json.dumps(data["metadata"]), data["embedding"]))
        conn.commit()
    else:
        collection.add(
            ids=[str(index)],
            embeddings=[data["embedding"]],
            metadatas=[data["metadata"]],
            documents=[str(row["lyrics"])]
        )
    # else:
        # print("The song already exist: {}".format(data["metadata"]["track_name"]))

In [ ]:
client = chromadb.Client()
# client.delete_collection(name="docs")
collection = client.create_collection(name="docs")

In [ ]:
for index, row in df.iterrows():
    try:
        addData2Table(index, cur, conn, row, collection ,is_add_to_table=True)
        print("{} Successfully add new song: {}".format(index, row["track_name"]))
    except Exception as e:
        print(e)
        print("failed to add {}".format(row["track_name"]))
        break

In [ ]:
# # Get all data from the collection
# chromadb_results = collection.get()

# # Export to JSON file
# with open('collection_data.json', 'w') as file:
#     json.dump(chromadb_results, file)

In [ ]:
# an example prompt
prompt = "Can you find the author and the release date of the song name 'dear heart'?"

# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
    prompt=prompt,
    model="nomic-embed-text"
)

results = collection.query(
    query_embeddings=response["embedding"],
    n_results=2
)
data_from_db = results
# print(len(data_from_db))
print(results["documents"][0])
# print(results)


In [ ]:
results["metadatas"][0]

In [ ]:
# generate a response combining the prompt and data we retrieved in step 2
filter_prompt = ollama.generate(
    model="llama3.1",
    prompt=f"""
    This is the user question: {prompt}
    
    Can you find the information from only this field in database: "track_name", "artist_name", "release_date", "len", "genre"
    
    Strictly answer either "YES" or "NO". If yes you need to specify the field that you can find this information. If the song is not exist, you must answer "NO". 
    """
)
print(filter_prompt['response'])

In [ ]:
# generate a response combining the prompt and data we retrieved in step 2
filter_prompt = ollama.generate(
    model="llama3.1",
    prompt=f"""
        From the list of metadata of 2 songs: {results["metadatas"][0]}

        Can you answer this question: {prompt}

        You must only choose the information from only one song from the list of metadata. 
        Do not mention the word "metadata" when answer the question, but you can use other key words from the list of metadata. Do not hallucinate.
    """
)
print(filter_prompt['response'])


In [ ]:
# generate a response combining the prompt and data we retrieved in step 2
summarize_lyrics1 = ollama.generate(
    model="llama3.1",
    # prompt=f"This is the prompt from the user: {data}. Respond using this information: {json.dumps(data_from_db)}. Only use the book from the database to answer.",
    prompt=f"""
    Summarize this random word using under 100 words: {results["documents"][0][0]}. 
    
    Do not write "Here's a summary", only write the content.
    """
)
print(summarize_lyrics1['response'])

# summarize_lyrics2 = ollama.generate(
#     model="llama3.1",
#     # prompt=f"This is the prompt from the user: {data}. Respond using this information: {json.dumps(data_from_db)}. Only use the book from the database to answer.",
#     prompt=f"""
#     Summarize this random word: {results["documents"][0][1]}. 
    
#     Do not write "Here's a summary", only write the content.
#     """
# )
# print(summarize_lyrics2['response'])

# summarize_selection = ollama.generate(
#     model="llama3.1",
#     # prompt=f"This is the prompt from the user: {data}. Respond using this information: {json.dumps(data_from_db)}. Only use the book from the database to answer.",
#     prompt=f"""
#     This is the user question: {prompt}

#     This is the meta data information from the song '{results["metadatas"][0][0]["track_name"]}': {json.dumps(results["metadatas"][0][0])}
    
#     This is the lyrics of the song '{results["metadatas"][0][0]["track_name"]}: {results["documents"][0][0]}


#     This is the meta data information of the song '{results["metadatas"][0][1]["track_name"]}: {json.dumps(results["metadatas"][0][1])}

#     This is the lyrics of the song '{results["metadatas"][0][1]["track_name"]}: {results["documents"][0][1]}

#     Answer the user question by selecting either information from '{results["metadatas"][0][0]["track_name"]}' or '{results["metadatas"][0][1]["track_name"]}' that are more closely relate to the user question. Why you select this song? 
#     If you think that both song are not related to user question. You must say that there is no song in the database that match the user description. 
#     Do not make up information. Do not mention the another song in the output. Do not use the sentence that means 'Based on the user's question' when answer. Do not mention the word that means summary.
#     """
# )
# print("__________________")
# print(summarize_selection['response'])

In [ ]:
# # Close the cursor and connection
# cur.close()
# conn.close()

# print("Data inserted successfully.")